In [39]:
from igraph import Graph
import matplotlib.pyplot as plt
import json

In [40]:
def read_adj_list_adjlist(filename: str) -> tuple[int, list[tuple[int, int]]]:
    edges = []
    with open(filename, 'r') as f:
        max_node = 0
        for line in f:
            if line.startswith("#") or line.strip() == "":
                continue
            parts = line.strip().split()
            src = int(parts[0])
            if len(parts[1:]) != 0:
                neighbors = map(int, parts[1:])
                for dst in neighbors:
                    edges.append((src, dst))
                    mn = max(neighbors)
            else:
                mn = 0
            max_node = max(max_node, src, mn)
    return max_node + 1, edges

def create_directed_graph_adjlist(filename: str) -> Graph:
    nb_vertex, edge_list = read_adj_list_adjlist(filename)
    g = Graph(directed=True)
    g.add_vertices(nb_vertex)
    g.add_edges(edge_list)

    return g

def create_real_graphs(filename: str) -> Graph:
    edge_list = []
    with open(filename, 'r') as f:
        for line in f:
            src, dst = map(int, line.strip().split(","))
            edge_list.append((src, dst))
    edges = [edge for edge in edge_list]
    g = Graph(directed=True)
    g.add_vertices(max(max(u, v) for u, v in edges) + 1)
    g.add_edges(edges)

    return g

In [41]:
g = create_directed_graph_adjlist("data/graph_10_100_2_0.5_0.3.adjlist")
print(g.summary())

IGRAPH D--- 100 100 -- 


In [42]:
g = create_directed_graph_adjlist("bdata/graph_10_1000_2_0.5_0.3.adjlist")
print(g.summary())

IGRAPH D--- 1000 1000 -- 


In [ ]:
with open("config.json", "r", encoding="utf-8") as jsonfile:
    data = json.load(jsonfile)
    print("Read successful")


def compute_wot_rank(trust_graph: Graph, truster: int, trustee: int) -> int:
    """
    Compute the rank (shortest path distance for positive edges only) from truster to trustee.
    """
    # Compute shortest path length

    rank = trust_graph.distances(source=truster, target=trustee)
    # except ig:
    #     rank = -1  # float('inf')  # No path found
    return rank[0][0]


def compute_wot_capacity(rank: int) -> int:
    """
    Compute the capacity based on the rank using the defined exponential decay formula.
    """
    if rank < 0:
        return 0
    # if rank >= 100:
    #     return 0 # infinite

    if rank > 5:
        return 1

    return data["rank_to_capacity"][rank]


def compute_wot_score(
    trust_graph: Graph, truster: int, trustee: int, verbose: bool = False
) -> int:
    """
    Compute the score of trustee from the perspective of truster.
    """
    if truster == trustee:
        return 100
    if trustee in trust_graph.neighbors(truster, mode='out'):
        # Direct trust value exists
        # return 100 #trust_graph[truster][trustee]['weight']

        return 100
    # Indirect computation

    score = 0
    neighbor_of_trustee = trust_graph.neighbors(trustee, mode='in')
    print("return of incident", neighbor_of_trustee)
    for neighbor in neighbor_of_trustee:
        trust_value = 100  # trust_graph[neighbor][trustee]['weight']
        rank = compute_wot_rank(trust_graph, truster, neighbor)
        capacity = compute_wot_capacity(rank)
        score += (trust_value * capacity) // 100

        if verbose:
            if rank >= 0:
                print(
                    f"  {truster}--...-->{neighbor}->{trustee}: rank={rank} capacity={capacity}"
                )
            else:
                print(
                    f"  {truster} NOPATH {neighbor}->{trustee}: rank={rank} capacity={capacity}"
                )
    # if score != 0:
    #     score = round(score / len(neighbor_of_trustee))

    return score

Read successful


In [44]:
g = create_real_graphs("rdata/graph4.csv")
print(g.summary())

IGRAPH D--- 907 238878 -- 


In [45]:
print(compute_wot_score(g, 1, 2))

return of incident [3, 5, 9, 15, 32, 35, 38, 39, 41, 46, 48, 55, 56, 64, 67, 69, 70, 71, 72, 83, 84, 93, 95, 98, 100, 102, 104, 105, 109, 124, 134, 136, 148, 149, 152, 157, 159, 161, 164, 166, 170, 171, 175, 176, 186, 196, 200, 203, 204, 208, 227, 231, 234, 237, 241, 246, 247, 248, 249, 270, 271, 275, 277, 278, 279, 282, 283, 289, 292, 293, 297, 302, 303, 309, 312, 314, 322, 325, 332, 336, 341, 342, 345, 346, 351, 365, 379, 380, 385, 389, 393, 397, 398, 406, 407, 413, 414, 415, 417, 424, 425, 433, 436, 437, 442, 444, 456, 461, 466, 470, 477, 479, 482, 484, 486, 489, 493, 496, 504, 508, 509, 523, 528, 530, 536, 537, 545, 546, 551, 552, 560, 565, 567, 569, 573, 576, 584, 587, 596, 609, 616, 625, 628, 629, 635, 640, 650, 651, 654, 661, 664, 665, 669, 680, 695, 697, 699, 705, 709, 718, 719, 720, 723, 741, 746, 749, 750, 752, 754, 758, 772, 775, 779, 785, 786, 787, 796, 813, 817, 829, 831, 836, 837, 838, 849, 863, 864]
4528
